In [1]:
import numpy as np
import pandas as pd
import os
import argparse
import errno
import scipy.misc
import dlib
import cv2
from skimage.feature import hog

ModuleNotFoundError: No module named 'dlib'

In [ ]:
print( "preparing")
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
original_labels = [0, 1, 2, 3, 4, 5, 6]

In [ ]:
def get_landmarks(image, rects):
    # this function have been copied from http://bit.ly/2cj7Fpq
    if len(rects) > 1:
        raise BaseException("TooManyFaces")
    if len(rects) == 0:
        raise BaseException("NoFaces")
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

In [ ]:
SELECTED_LABELS = [0,1,2,3,4,5,6]
new_labels = list(set(original_labels) & set(SELECTED_LABELS))
nb_images_per_label = list(np.zeros(len(new_labels), 'uint8'))
def get_new_label(label, one_hot_encoding=False):
    if one_hot_encoding:
        new_label = new_labels.index(label)
        label = list(np.zeros(len(new_labels), 'uint8'))
        label[new_label] = 1
        return label
    else:
        return new_labels.index(label)

In [ ]:
print( "importing csv file")
data = pd.read_csv('fer2013.csv')

In [ ]:
print(data['Usage'].unique())
image_height = 48
image_width = 48
window_size = 24
window_step = 6

for category in data['Usage'].unique():
    print( "converting set: " + category + "...")
    if not os.path.exists(category):
        try:
            os.makedirs(category)
        except OSError as e:
            print("error!: ", e)
    
    # get samples and labels of the actual category
    category_data = data[data['Usage'] == category]
    samples = category_data['pixels'].values
    labels = category_data['emotion'].values
    
    # get images and extract features
    images = []
    labels_list = []
    landmarks = []
    hog_features = []
    hog_images = []
    
    for i in range(len(samples)):
        try:
            image = np.fromstring(samples[i], dtype=int, sep=" ").reshape((image_height, image_width))
            images.append(image)
            scipy.misc.imsave('temp.jpg', image)
            image2 = cv2.imread('temp.jpg')
            
            '''Extracting face landmarks here'''
            face_rects = [dlib.rectangle(left=1, top=1, right=47, bottom=47)]
            face_landmarks = get_landmarks(image2, face_rects)
            landmarks.append(face_landmarks) 
            labels_list.append(get_new_label(labels[i], one_hot_encoding=True))
            nb_images_per_label[get_new_label(labels[i])] += 1
            
            features, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                                            cells_per_block=(1, 1), visualise=True)
            hog_features.append(features)
            hog_images.append(hog_image)
        except Exception as e:
            print( "error in image: " + str(i) + " - " + str(e))
    
    np.save("./"+category+'/images.npy', images)
    np.save("./"+category+'/landmarks.npy', landmarks)
    np.save("./"+category+'/labels.npy', labels_list)
    np.save("./"+category+'/hog_features.npy', hog_features)
    np.save("./"+category+'/hog_images.npy', hog_images)

In [ ]:
data_dict = dict()
validation_dict = dict()
test_dict = dict()

In [ ]:
data_dict['X'] = np.load('./Training' + '/landmarks.npy')
data_dict['X'] = np.array([x.flatten() for x in data_dict['X']])
data_dict['X'] = np.concatenate((data_dict['X'], np.load('./Training' + '/hog_features.npy')), axis=1)

In [ ]:
data_dict['Y'] = np.load('./Training' +  '/labels.npy')

In [ ]:
validation_dict['X'] = np.load('./PublicTest' + '/landmarks.npy')
validation_dict['X'] = np.array([x.flatten() for x in validation_dict['X']])
validation_dict['Y'] = np.load('./PublicTest'  + '/labels.npy')
validation_dict['X'] = np.concatenate((validation_dict['X'], np.load('./PublicTest' + '/hog_features.npy')), axis=1)

In [ ]:
test_dict['X'] = np.load('./PrivateTest' + '/landmarks.npy')
test_dict['X'] = np.array([x.flatten() for x in test_dict['X']])
test_dict['Y'] = np.load('./PrivateTest'  + '/labels.npy')
test_dict['X'] = np.concatenate((test_dict['X'], np.load('./PrivateTest' + '/hog_features.npy')), axis=1)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

In [ ]:
print( "building model...")
#model = SVC(random_state=42, max_iter=300)
model = SVC(random_state=42, max_iter=10000, kernel='linear', gamma=0.0001, decision_function_shape = 'ovr')
print("done")

In [ ]:
print("Training")
start_time = time.time()
dict_y=np.ndarray(len(data_dict['Y']))
count=0
for row in range(len(data_dict['Y'])):
    temp = data_dict['Y'][row][0]
    dict_y[count] = temp
    count+=1
model.fit(data_dict['X'], dict_y)
training_time = time.time() - start_time
print( "training time = {0:.1f} sec".format(training_time))

In [ ]:
def evaluate(model, X, Y):
    predicted_Y = model.predict(X)
    accuracy = accuracy_score(Y, predicted_Y)
    return accuracy


In [ ]:
dict_v=np.ndarray(len(validation_dict['Y']))
count=0
for row in range(len(validation_dict['Y'])):
    temp = validation_dict['Y'][row][0]
    dict_v[count] = temp
    count+=1

In [ ]:
validation_accuracy = evaluate(model, validation_dict['X'], dict_v)
print( "  - validation accuracy = {0:.1f}".format(validation_accuracy*100))

In [ ]:
dict_t=np.ndarray(len(test_dict['Y']))
count=0
for row in range(len(test_dict['Y'])):
    temp = test_dict['Y'][row][0]
    dict_t[count] = temp
    count+=1

In [ ]:
test_accuracy = evaluate(model, test_dict['X'], dict_t)
print( "  - test accuracy = {0:.1f}".format(test_accuracy*100))

In [ ]:
'''Using ONLY HOG features'''
data_hog_train = np.load('./Training' + '/hog_features.npy')

In [ ]:
start = time.time()
model.fit(data_hog_train, dict_y)
training_time = time.time() - start_time
print( "training time = {0:.1f} sec".format(training_time))

In [ ]:
data_validation_hog = np.load('./PublicTest' + '/hog_features.npy')
validation_accuracy = evaluate(model, data_validation_hog, dict_v)
print( "  - validation accuracy = {0:.1f}".format(validation_accuracy*100))

In [ ]:
data_test_hog = np.load('./PrivateTest' + '/hog_features.npy')
test_accuracy = evaluate(model, data_test_hog, dict_t)
print( "  - test accuracy = {0:.1f}".format(test_accuracy*100))

In [ ]:
'''Adding Training + Valid '''
data_dict['X'] = np.load('./Training' + '/landmarks.npy')
data_dict['X'] = np.array([x.flatten() for x in data_dict['X']])
validation_dict['X'] = np.load('./PublicTest' + '/landmarks.npy')
validation_dict['X'] = np.array([x.flatten() for x in validation_dict['X']])
data_dict['X'] = np.concatenate((data_dict['X'], validation_dict['X']), axis=0)

In [ ]:
dict_y = np.concatenate((dict_y, dict_v), axis = 0)
model.fit(data_dict['X'], dict_y)
training_time = time.time() - start_time
print( "training time = {0:.1f} sec".format(training_time))

In [ ]:
test_dict['X'] = np.load('./PrivateTest' + '/landmarks.npy')
test_dict['X'] = np.array([x.flatten() for x in test_dict['X']])
test_accuracy = evaluate(model, test_dict['X'], dict_t)
print( "  - test accuracy = {0:.1f}".format(test_accuracy*100))

In [ ]:
'''Only Face Landmarks'''

data_dict['X'] = np.load('./Training' + '/landmarks.npy')
data_dict['X'] = np.array([x.flatten() for x in data_dict['X']])

validation_dict['X'] = np.load('./PublicTest' + '/landmarks.npy')
validation_dict['X'] = np.array([x.flatten() for x in validation_dict['X']])

test_dict['X'] = np.load('./PrivateTest' + '/landmarks.npy')
test_dict['X'] = np.array([x.flatten() for x in test_dict['X']])

In [ ]:
model = SVC(random_state=42, max_iter=10000, kernel='linear', gamma=0.0001, decision_function_shape = 'ovr')

In [ ]:
data_dict['Y'] = np.load('./Training' +  '/labels.npy')
dict_y = np.ndarray(len(data_dict['Y']))
count=0
for row in range(len(data_dict['Y'])):
    temp = data_dict['Y'][row][0]
    dict_y[count] = temp
    count+=1

In [ ]:
start = time.time()
model.fit(data_dict['X'], dict_y)
training_time = time.time() - start_time
print( "training time = {0:.1f} sec".format(training_time))

In [ ]:
test_dict['Y'] = np.load('./PrivateTest'  + '/labels.npy')
dict_t=np.ndarray(len(test_dict['Y']))
count=0
for row in range(len(test_dict['Y'])):
    temp = test_dict['Y'][row][0]
    dict_t[count] = temp
    count+=1

In [ ]:
test_accuracy = evaluate(model, test_dict['X'], dict_t)
print("  - test accuracy = {0:.1f}%".format(test_accuracy*100))
print("  - training accuracy:  = {0:.1f}%".format(model.score(data_dict['X'], dict_y)*100))

In [ ]:
validation_dict['Y'] = np.load('./PrivateTest'  + '/labels.npy')
dict_t=np.ndarray(len(validation_dict['Y']))
count=0
for row in range(len(validation_dict['Y'])):
    temp = validation_dict['Y'][row][0]
    dict_t[count] = temp
    count+=1
validation_accuracy = evaluate(model, validation_dict['X'], dict_v)
print("  - validation accuracy = {0:.1f}%".format(validation_accuracy*100))